# Training Part

In [ ]:
import tensorflow as tf

<h3> LoadingImages </h3>

In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split

# Load the CSV file
data = pd.read_csv(r'D:\Dev\OfficialProjects\AnemiaDetection\NewDataWorkspace\conjunctiva\metadataConjuctiva.csv')

# Define image size and paths
image_size = (128, 128)
image_folder = r'D:\Dev\OfficialProjects\AnemiaDetection\NewDataWorkspace\conjunctiva\DATA'

# Function to load and preprocess images
def load_images(data, image_folder, image_size):
    images = []
    labels = []
    for index, row in data.iterrows():
        img_path = os.path.join(image_folder, row['F_name'])
        image = cv2.imread(img_path)
        if image is not None:
            image = cv2.resize(image, image_size)
            image = image / 255.0  # Normalize to [0, 1]
            images.append(image)
            labels.append(row['anemic'])
    return np.array(images), np.array(labels)

images, labels = load_images(data, image_folder, image_size)


In [ ]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

def create_model(input_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling2D((2, 2)),

        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),

        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),

        Conv2D(256, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),

        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')  # Assuming binary classification
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


# Define input shape
input_shape = (image_size[0], image_size[1], 3)

# Create models
model = create_model(input_shape)

In [ ]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

In [ ]:
model.predict(X_val)

In [ ]:
model.save(filepath=r"D:\Dev\OfficialProjects\AnemiaDetection\Model\conjunctiva.h5")

# Prediction Part

In [7]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

def predict_anemia(model, img_path, image_size=(128, 128)):
    # Load and preprocess the image
    img = cv2.imread(img_path)
    if img is not None:
        img = cv2.resize(img, image_size)
        img = img / 255.0  # Normalize to [0, 1]
        img_array = np.expand_dims(img, axis=0)  # Add batch dimension

        # Make a prediction
        prediction = model.predict(img_array)

        # Convert the prediction to a binary class label
        threshold = 0.5
        class_label = (prediction > threshold).astype(int)

        # Output the result
        if class_label == 0:
            return "Not Anemic"
        else:
            return "Anemic"
    else:
        return "Image could not be loaded"

# Example usage:

img_path = r'D:\Dev\OfficialProjects\AnemiaDetection\NewDataWorkspace\conjunctiva\DATA\Non-Anemic-052 (2).png'  # Update this path

fingerNail__model = load_model(r"D:\Dev\OfficialProjects\AnemiaDetection\Model\conjunctiva_100.h5")

# Predict using the model
result = predict_anemia(fingerNail__model, img_path)
print(result)


1/1 [==============================] - 0s 150ms/step
Not Anemic
